In [15]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

In [16]:
url = f'https://kin.naver.com/search/list.naver?query={quote("부동산 월세")}&page=1'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(3)

The chromedriver version (117.0.5938.88) detected in PATH at C:\Users\human\Downloads\chromedriver.exe might not be compatible with the detected chrome version (118.0.5993.89); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [17]:
lis = soup.select('.basic1 > li')
len(lis)

10

In [18]:
li = lis[6]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=60202&docId=438009893&qb=67aA64+Z7IKwIOyblOyEuA==&enc=utf8&section=kin&rank=7&search_sort=0&spq=0'

In [19]:
driver.get(sub_url)
time.sleep(2)

In [20]:
sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
questions = sub_soup.select('#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content')
divs = sub_soup.select('#answer_1 > div._endContents.c-heading-answer__content > div._endContentsText.c-heading-answer__content-user > div > div')
len(questions),len(divs)

(1, 1)

In [7]:
answer = ''
for p in questions:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += text
print(answer + '\t' + 'hu')

마음에 드는 원룸이 보증금 5천 월세20 관리비 10입니다최우선변제금액이 3400만원이구요건물시세가 밸류맵으로 확인해봤을때 7억8천8백만원이였구요 등기부등본에 채권최고액이 5억4천으로공동담보로 잡혀있었습니다안전한 집 일까요…..	hu


In [8]:
div = divs[0]
ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
len(ps)

4

In [9]:
answer = ''
for p in ps:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += text
print(answer)

상태로 보았을때 안전하지 않습니다.혹시 모를 위험애 대비해서 다른곳을 찾는걸 추천 드립니다..궁금증이 풀리셨다면 채택 부탁드립니다!


In [21]:
data_list = []
driver = webdriver.Chrome()
for page in range(1, 36):
    url = f'https://kin.naver.com/search/list.naver?query={quote("부동산 월세")}&page={page}'
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        questions = sub_soup.select('#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content')
        divs = sub_soup.select('#answer_1 > div._endContents.c-heading-answer__content > div._endContentsText.c-heading-answer__content-user > div > div')
        
        answer = ''
        for p in questions:
            text = p.get_text().strip()
            answer += text
        answer1 = ''
        for p in divs:
            text = p.get_text().strip()
            answer1 += text
        
        data_list.append({'구분': '월세', '질문': answer, '답변': answer1})
         
        driver.back()
        time.sleep(2)
    print()

driver.close()

The chromedriver version (117.0.5938.88) detected in PATH at C:\Users\human\Downloads\chromedriver.exe might not be compatible with the detected chrome version (118.0.5993.89); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [22]:
df = pd.DataFrame(data_list)
df

,구분,질문,답변
0,월세,"1.,부동산 월세 계약서를 집주인분과 협의 및 수정하는데법적으로 문제가될까요?2.연...","1.,부동산 월세 계약서를 집주인분과 협의 및 수정하는데법적으로 문제가될까요?<답변..."
1,월세,"안녕하세요?신탁 오피스텔 임대차 계약(보증금 5천만원, 월세 30만원)을 하려고 합...",안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 정종욱 변호...
2,월세,오피스텔 주인으로서 월세 다른사람 한테 월세 받고 잇는 상황인데 세입자가 계약후 퇴...,세입자의 과실이면 원상복구의 책임이 있습니다. ​이런 경우 인테리어 업체에서 견적...
3,월세,"안녕하세요저는 집주인이고, 월세 계약을 체결했습니다.현재 기준, 입주 약 1개월 전...","안녕하세요. 고양 일산 파주 김포, 부동산 월세 계약 파기, 계약금 반환 문의, 율..."
4,월세,전입신고 안한 오피스텔 거주중입니다10월이 2년만기인데 월세 55만원을 내고 거주중...,
...,...,...,...
345,월세,3월 29일에 부동산 계약서를 작성하고 왔습니다. 계약서상 입주일은 4월 30일이고...,질문에 답변드리겠습니다.1. 상기 질문요지 및 보증금 규모를 고려할 때 최우선 변제...
346,월세,제가 부동산을 끼고 몇주 전 원룸을 월세로 계약했습니다. 그런데 오늘 일방적으로 집...,오늘 일방적으로 집주인이 계약을 파기 하게 되었는데 이럴경우 제가 부동산에 지불한 ...
347,월세,제 와이프가 부업을 한다고 다세대 주택 방3칸짜리 집을 보증금 500에 월세40만원...,먼저 월세를 계속 안내게 되서 보증금이 모두 소진되면 끝이 납니다.더 이상 안내도 ...
348,월세,자취를 하려고 부동산을 통해 월세방을 알아보고 오늘 계약을 했거든요. 근데 제가 자...,


In [23]:
df.to_csv('data/부동산 월세.csv', index=False, encoding='utf-8')